In [1]:
pip install plotly panel

   ---------------------------------------- 0.0/17.3 MB ? eta -:--:--
    --------------------------------------- 0.3/17.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/17.3 MB 1.4 MB/s eta 0:00:12
   -- ------------------------------------- 1.0/17.3 MB 2.0 MB/s eta 0:00:09
   --- ------------------------------------ 1.6/17.3 MB 2.1 MB/s eta 0:00:08
   ---- ----------------------------------- 2.1/17.3 MB 2.1 MB/s eta 0:00:08
   ------ --------------------------------- 2.6/17.3 MB 2.3 MB/s eta 0:00:07
   ------- -------------------------------- 3.4/17.3 MB 2.4 MB/s eta 0:00:06
   --------- ------------------------------ 4.2/17.3 MB 2.5 MB/s eta 0:00:06
   ---------- ----------------------------- 4.7/17.3 MB 2.5 MB/s eta 0:00:05
   ------------ --------------------------- 5.2/17.3 MB 2.6 MB/s eta 0:00:05
   ------------- -------------------------- 6.0/17.3 MB 2.7 MB/s eta 0:00:05
   --------------- ------------------------ 6.6/17.3 MB 2.7 MB/s eta 0:00:05
   ----------

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

In [19]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('combined_transactions_categories.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

,Date,Name / Description,Expense/Income,Amount,Category,Year,Month,Month Name
0,07-31-2024,AplPay APPLE.COM/BILINTERNET CHARGE CA,Expense,30.21,Technology,2024,7,Jul
1,07-31-2024,BH* BETTERHELP MOUNTAIN VIEW CA,Expense,360.00,Internet Service Provider,2024,7,Jul
2,07-31-2024,LYFT 855-280-0278 CA,Expense,25.84,Mental Health,2024,7,Jul
3,07-31-2024,LYFT 855-280-0278 CA,Expense,33.95,Transportation,2024,7,Jul
4,07-30-2024,AplPay STANZA-VINYL San Francisco CA,Expense,9.75,Pharmacy,2024,7,Jul
...,...,...,...,...,...,...,...,...
135,08-01-2024,AplPay TRADER JOE'S SAN FRANCISCO CA,Expense,31.93,NaN,2024,8,Aug
136,08-01-2024,LYFT 855-280-0278 CA,Expense,9.59,NaN,2024,8,Aug
137,07-01-2024,AplPay JOE & THE JUICE BURLINGAME CA,Expense,13.37,NaN,2024,7,Jul
138,07-01-2024,AplPay PIROO San Francisco CA,Expense,38.72,NaN,2024,7,Jul


In [24]:
import pandas as pd
import plotly.express as px

def make_pie_chart(df, year, label):
    # Filter the dataset for the specified expense/income and year
    sub_df = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]

    # Remove NaN values from the 'Category' column
    sub_df = sub_df.dropna(subset=['Category'])

    # Check if the filtered dataframe is empty after removing NaN values
    if sub_df.empty:
        raise ValueError(f"No data available for {label} in year {year}")

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount', names='Category', color_discrete_sequence=color_scale)
    pie_fig.update_traces(textposition='inside', direction='clockwise', hole=0.3, textinfo="label+percent")

    # Ensure Amount (EUR) is numeric
    sub_df['Amount'] = pd.to_numeric(sub_df['Amount'], errors='coerce')
    total_expense = sub_df['Amount'].sum()

    if pd.isna(total_expense):
        raise ValueError("Total expense calculation returned NaN")

    total_text = "€ " + str(round(total_expense))

    pie_fig.update_layout(
        uniformtext_minsize=10,
        uniformtext_mode='hide',
        title=dict(text=label + " Breakdown " + str(year)),
        annotations=[
            dict(
                text=total_text,
                x=0.5, y=0.5, font_size=12,
                showarrow=False
            )
        ]
    )

    return pie_fig

# Example usage
try:
    expense_pie_fig_2024 = make_pie_chart(df, 2024, 'Expense')
    expense_pie_fig_2024.show()  # Use .show() to display the figure
except Exception as e:
    print(f"Error: {e}")


Error: Mime type rendering requires nbformat>=4.2.0 but it is not installed


In [25]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount', text_auto='.2s', title=label+" per month", color='Amount', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [26]:
expense_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Expense')
expense_monthly_2024

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Month Name=%{x}<br>Amount=%{marker.color}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': array([ 646.25, 2302.48, 3962.25,   97.15]),
                         'coloraxis': 'coloraxis',
                         'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'texttemplate': '%{y:.2s}',
              'type': 'bar',
              'x': array(['May', 'Jun', 'Jul', 'Aug'], dtype=object),
              'xaxis': 'x',
              'y': array([ 646.25, 2302.48, 3962.25,   97.15]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'coloraxis': {'colorbar': {'title': {'text': 'Amount'}},
                             'colorscale': [[0.0, 'rgb(255,247,236)'], [0.125,
                                            'rgb(254,232,200)'], [0.25,
                                            'rgb(253,212,158)'], [0.375,
                                            'rgb(253,187,132)'], [0.5,
                                            'rgb(252,141,89)'], [0.625,
                                            'rgb(239,101,72)'], [0.75,
                                            'rgb(215,48,31)'], [0.875,
                                            'rgb(179,0,0)'], [1.0, 'rgb(127,0,0)']]},
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Expense per month'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Month Name'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Amount'}}}
})

In [27]:
# Creating the charts
expense_pie_fig_2024 = make_pie_chart(df, 2024, 'Expense')
expense_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Expense')

# Create tabs
tabs = pn.Tabs(
    ('2024', pn.Column(pn.Row(expense_monthly_2024, expense_pie_fig_2024)))
)

# Display the tabs
tabs.show()

Launching server at http://localhost:52163
